In [1]:
import sys
sys.path.append('../')
from mytapering import *

In [2]:
import pennylane as qml
from pennylane import qchem

import numpy as np
from time import time

In [3]:
np.set_printoptions(precision=6)

# 4. Example: BeH2

In [4]:
# http://ursula.chem.yale.edu/~chem220/chem220js/STUDYAIDS/hybridization/mo/beh2.log
symbols = ["Be", "H", "H"]
coordinates = np.array([-0.49317, 1.16085, 0.0, 0.69683, 1.16085, 0.0, -1.68317, 1.16085, 0.0])

In [5]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  14
The Hamiltonian is 
   (-6.311794785724469) [I0]
+ (-0.8171638849443204) [Z13]
+ (-0.8171638849443201) [Z12]
+ (-0.3412642728703804) [Z10]
+ (-0.3412642728703803) [Z11]
+ (-0.07268403369472537) [Z7]
+ (-0.07268403369472531) [Z6]
+ (-0.07268403369472362) [Z9]
+ (-0.07268403369472358) [Z8]
+ (0.08400532522364187) [Z3]
+ (0.08400532522364221) [Z2]
+ (0.09592687719026514) [Z4]
+ (0.09592687719026514) [Z5]
+ (2.4673468849172937) [Z0]
+ (2.4673468849172946) [Z1]
+ (0.07717080613357652) [Z10 Z12]
+ (0.07717080613357652) [Z11 Z13]
+ (0.08687237248084279) [Z2 Z4]
+ (0.08687237248084279) [Z3 Z5]
+ (0.08998679138300451) [Z4 Z6]
+ (0.08998679138300451) [Z5 Z7]
+ (0.08998679138302458) [Z4 Z8]
+ (0.08998679138302458) [Z5 Z9]
+ (0.09124508190558271) [Z6 Z10]
+ (0.09124508190558271) [Z7 Z11]
+ (0.09124508190560301) [Z8 Z10]
+ (0.09124508190560301) [Z9 Z11]
+ (0.09150767112431096) [Z6 Z12]
+ (0.09150767112431096) [Z7 Z13]
+ (0.09150767112433156) [Z8 Z12]
+ (0.09150767112433156) [

## 4.1 Tapering using QML

In [6]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 6
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

C:\Users\Lee\anaconda3\envs\qiskit\lib\site-packages\pennylane\operation.py:1921: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
C:\Users\Lee\anaconda3\envs\qiskit\lib\site-packages\pennylane\operation.py:2111: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


time cost: 58.72396373748779
Number of qubit after tapering = 9
The tapered Hamiltonian is 
   ((-6.086865240604816-9.540979117872439e-18j)) [I0]
+ ((-0.34126427287038+1.734723475976807e-18j)) [Z10]
+ ((-0.14536806738945063+6.071532165918825e-17j)) [Z6]
+ ((-0.1453680673894471+5.898059818321144e-17j)) [Z8]
+ ((-0.024996812058313084+0j)) [X5]
+ ((0.00239851193788527+0j)) [X6]
+ ((0.00239851193788583+0j)) [X8]
+ ((0.0840053252236418+0j)) [Z3]
+ ((0.08400532522364214+0j)) [Z2]
+ ((0.09592687719026513-7.806255641895632e-18j)) [Z4]
+ ((0.09592687719026513+0j)) [Z5]
+ ((2.467346884917291-6.938893903907228e-18j)) [Z0]
+ ((2.467346884917292+6.938893903907228e-18j)) [Z1]
+ ((-0.0437109433902769+0j)) [Y2 Y3]
+ ((-0.02198098982547869+0j)) [Y4 Y10]
+ ((-0.02198098982547869+0j)) [X4 X10]
+ ((-0.015932894580459225+0j)) [X5 X10]
+ ((-0.015276353816802757-8.673617379884035e-19j)) [X5 Z10]
+ ((-0.011747610873581945+0j)) [Y4 Y5]
+ ((-0.0008727725039831971+0j)) [Y0 Y1]
+ ((0.00018149288502513687-0j)) [Y1

In [7]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, 11)

-6.086865240605 IIIIIIIIIII
 0.817163884944 ZIIZZIZIZIZ
 0.817163884944 ZIZIZIZIZIZ
 0.255944263715 IIZZIIIIIII
-0.341264272870 ZZIIZZIIIIZ
-0.077170806134 IZIZIZZIZII
-0.115811470765 IZZIIZZIZII
-0.145368067389 IIIIIIIIZII
-0.185413854187 ZIIZZIZIIIZ
-0.185413854187 ZIZIZIZIIIZ
 0.190380873170 ZZIIZZIIZIZ
-0.145368067389 IIIIIIZIIII
-0.185413854187 ZIIZZIIIZIZ
-0.185413854187 ZIZIZIIIZIZ
 0.190380873170 ZZIIZZZIIIZ
 0.389235633557 IIIIIIZIZII
 2.467346884917 ZIIIIIIIIII
-0.131729034034 IIIZZIZIZIZ
-0.130856261530 IIZIZIZIZIZ
 0.145944382624 IZIIZZIIIIZ
 0.280331498970 ZIIIIIIIZII
 0.280331498970 ZIIIIIZIIII
 2.467346884917 IZIIIIIIIII
-0.130856261530 ZZIZZIZIZIZ
-0.131729034034 ZZZIZIZIZIZ
 0.138874112270 ZIIIZZIIIIZ
 0.280331498970 IZIIIIIIZII
 0.280331498970 IZIIIIZIIII
 0.086872372481 IIIZIZIIIII
 0.123744329956 IIZIIZIIIII
 0.559461224125 ZZIIIIIIIII
 0.067832693201 YZZZYIIIIII
 0.000213756392 XZZIXIZIZIZ
 0.002247215843 XZIZXIZIZIZ
-0.002738112253 XIZZXZIIIIZ
 0.010448181277 YZZZ

## 4.2 Tapering using mytarpering

In [8]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-6.311794785724 IIIIIIIIIIIIII
 2.467346884917 ZIIIIIIIIIIIII
 2.467346884917 IZIIIIIIIIIIII
 0.559461224125 ZZIIIIIIIIIIII
 0.067832693201 YZZZYIIIIIIIII
 0.078415863800 YIZZYIIIIIIIII
 0.067832693201 XZZZXIIIIIIIII
 0.078415863800 XIZZXIIIIIIIII
 0.048277063636 YZZZZZZZZZYIII
 0.043293922289 YIZZZZZZZZYIII
 0.048277063636 XZZZZZZZZZXIII
 0.043293922289 XIZZZZZZZZXIII
 0.004668976323 YXXYIIIIIIIIII
-0.004668976323 YYXXIIIIIIIIII
-0.004668976323 XXYYIIIIIIIIII
 0.004668976323 XYYXIIIIIIIIII
-0.000163938150 YXIXZZZZZZZZYI
-0.000163938150 YYIYZZZZZZZZYI
-0.000163938150 XXIXZZZZZZZZXI
-0.000163938150 XYIYZZZZZZZZXI
 0.067832693201 IYZZZYIIIIIIII
 0.078415863800 ZYZZZYIIIIIIII
 0.067832693201 IXZZZXIIIIIIII
 0.078415863800 ZXZZZXIIIIIIII
 0.018021322949 YXIIXYIIIIIIII
-0.018021322949 YYIIXXIIIIIIII
-0.018021322949 XXIIYYIIIIIIII
 0.018021322949 XYIIYXIIIIIIII
-0.010764826109 YXIIIXZZZZYIII
-0.010764826109 YYIIIYZZZZYIII
-0.010764826109 XXIIIXZZZZXIII
-0.010764826109 XYIIIYZZZZXIII
 0.00396

In [ ]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

## 4.3 compare

### 4.3.1 generators and paulixops

In [ ]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

In [ ]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

### 4.3.2 optimal sectors

In [ ]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')